In [67]:
from pathlib import Path

estimator_path = '/Users/vobornij/projects/polymarket/data/estimator/2025-09-03/bitcoin-up-or-down-september-3-10am-et_0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10.jsonl'

market = Path(estimator_path).name.rsplit('_', 1)[-1].removesuffix('.jsonl')
slug = Path(estimator_path).name.rsplit('_', 1)[0]
print(f'market: {market} slug: {slug}')

market: 0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10 slug: bitcoin-up-or-down-september-3-10am-et


In [68]:
import pathlib, json, time, datetime, requests
from typing import List, Dict, Any, Optional

MARKET_ID = market

BASE_URL = "https://data-api.polymarket.com/trades"  # per docs
MAX_LIMIT = 500  # API max
OUTPUT_DIR = pathlib.Path('/Users/vobornij/projects/polymarket/data/trades')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def fetch_trades_batch(market: str, limit: int = MAX_LIMIT, offset: int = 0, taker_only: bool = True) -> List[Dict[str, Any]]:
    params = {
        'market': market,
        'limit': min(limit, MAX_LIMIT),
        'offset': offset,
        'takerOnly': str(taker_only).lower(),  # 'true' / 'false'
    }
    r = requests.get(BASE_URL, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not isinstance(data, list):
        raise ValueError(f'Unexpected response type: {type(data)} | {data}')
    return data

base = f"{market[:16]}"
jsonl_path = OUTPUT_DIR / f"{base}.jsonl"

def download_all_trades(market: str, batch_limit: int = MAX_LIMIT, sleep: float = 0.1, max_batches: Optional[int] = None, taker_only: bool = True) -> pathlib.Path:
    total = 0
    offset = 0
    batch_no = 0
    with jsonl_path.open('w') as f:
        while True:
            batch_no += 1
            batch = fetch_trades_batch(market, limit=batch_limit, offset=offset, taker_only=taker_only)
            if not batch:
                break
            for tr in batch:
                f.write(json.dumps(tr) + '\n')
            got = len(batch)
            print(f"Batch {batch_no}: fetched {got} trades (offset {offset}) at {datetime.datetime.now().isoformat()}")
            total += got
            offset += got
            # Stop conditions
            if got < batch_limit:
                # Last page
                break
            if max_batches and batch_no >= max_batches:
                break
            if sleep:
                time.sleep(sleep)
    summary = {
        'market': market,
        'file': jsonl_path.name,
        'total_trades': total,
        'batches': batch_no,
        'note': 'Trades returned newest-first (descending timestamp). Offset pagination accumulates all until exhaustion.'
    }
    (OUTPUT_DIR / f"{base}_summary.json").write_text(json.dumps(summary, indent=2))
    print(f"Saved {total} trades across {batch_no} batch(es) -> {jsonl_path}")
    return jsonl_path


# download trades if file does not exist: 
if not jsonl_path.exists():
    download_all_trades(MARKET_ID)


In [69]:
# Load latest downloaded trades for the current `market` into a DataFrame (asset forced to string)
import pandas as pd, pathlib, json, datetime, os
from typing import List, Dict, Any

MARKET_ID = globals().get('market')
if not MARKET_ID:
    raise ValueError("`market` variable not defined. Set it before running this cell.")

trades_dir = pathlib.Path('/Users/vobornij/projects/polymarket/data/trades')
prefix = MARKET_ID[:16] + '_'


print(f'Loading trades from {jsonl_path}')

# Manually parse JSONL to control dtypes and preserve very large identifiers as strings
rows: List[Dict[str, Any]] = []
with jsonl_path.open() as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        # Force asset to string exactly as in file (avoid pandas numeric inference / float formatting)
        if 'asset' in obj:
            obj['asset'] = str(obj['asset'])
        rows.append(obj)

df = pd.DataFrame(rows)

# # Ensure pandas didn't coerce asset
# if 'asset' in df.columns:
#     df['asset'] = df['asset'].astype('string')  # pandas string dtype retains exact text

# Add a human-readable UTC datetime column if timestamp present (assumes seconds)
if 'timestamp' in df.columns:
    def _to_dt(x):
        try:
            return datetime.datetime.fromtimestamp(int(x), datetime.timezone.utc)
        except Exception:
            return pd.NaT
    df['dt'] = df['timestamp'].map(_to_dt)

# Sort ascending by timestamp if present
if 'timestamp' in df.columns:
    df.sort_values('timestamp', inplace=True)

print(f'Trades loaded: {len(df)} rows')
if 'asset' in df.columns:
    sample_asset = df['asset'].iloc[0]
    print(f'asset dtype: {df['asset'].dtype}; sample asset length: {len(sample_asset)}')

df.head(5)

Loading trades from /Users/vobornij/projects/polymarket/data/trades/0xa7467e80e2dcb9.jsonl
Trades loaded: 245500 rows
asset dtype: object; sample asset length: 77


,proxyWallet,side,asset,conditionId,size,price,timestamp,title,slug,icon,eventSlug,outcome,outcomeIndex,name,pseudonym,bio,profileImage,profileImageOptimized,transactionHash,dt
245499,0x8749194e5105c97c3d134e974e103b44eea44ea4,BUY,99177944206005769990419225999420914922292092608088554323866698392058208125838,0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10,10.0,0.7,1756908302,"Bitcoin Up or Down - September 3, 10AM ET",bitcoin-up-or-down-september-3-10am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-september-3-10am-et,Up,0,0x066423fcbd2e38a7b5f4ba813eb447b823c91f,Doting-Fiction,,,,0x48671d8cc40f1d1cd9fe227f6d7858cb25a4e0fc73b55b70d5616a91505e6ac6,2025-09-03 14:05:02+00:00
158999,0x8749194e5105c97c3d134e974e103b44eea44ea4,BUY,99177944206005769990419225999420914922292092608088554323866698392058208125838,0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10,10.0,0.7,1756908302,"Bitcoin Up or Down - September 3, 10AM ET",bitcoin-up-or-down-september-3-10am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-september-3-10am-et,Up,0,0x066423fcbd2e38a7b5f4ba813eb447b823c91f,Doting-Fiction,,,,0x48671d8cc40f1d1cd9fe227f6d7858cb25a4e0fc73b55b70d5616a91505e6ac6,2025-09-03 14:05:02+00:00
90999,0x8749194e5105c97c3d134e974e103b44eea44ea4,BUY,99177944206005769990419225999420914922292092608088554323866698392058208125838,0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10,10.0,0.7,1756908302,"Bitcoin Up or Down - September 3, 10AM ET",bitcoin-up-or-down-september-3-10am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-september-3-10am-et,Up,0,0x066423fcbd2e38a7b5f4ba813eb447b823c91f,Doting-Fiction,,,,0x48671d8cc40f1d1cd9fe227f6d7858cb25a4e0fc73b55b70d5616a91505e6ac6,2025-09-03 14:05:02+00:00
73499,0x8749194e5105c97c3d134e974e103b44eea44ea4,BUY,99177944206005769990419225999420914922292092608088554323866698392058208125838,0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10,10.0,0.7,1756908302,"Bitcoin Up or Down - September 3, 10AM ET",bitcoin-up-or-down-september-3-10am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-september-3-10am-et,Up,0,0x066423fcbd2e38a7b5f4ba813eb447b823c91f,Doting-Fiction,,,,0x48671d8cc40f1d1cd9fe227f6d7858cb25a4e0fc73b55b70d5616a91505e6ac6,2025-09-03 14:05:02+00:00
143499,0x8749194e5105c97c3d134e974e103b44eea44ea4,BUY,99177944206005769990419225999420914922292092608088554323866698392058208125838,0xa7467e80e2dcb9786cde7fdf754d29172c7d0ea7935566e2da118593f9406a10,10.0,0.7,1756908302,"Bitcoin Up or Down - September 3, 10AM ET",bitcoin-up-or-down-september-3-10am-et,https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png,bitcoin-up-or-down-september-3-10am-et,Up,0,0x066423fcbd2e38a7b5f4ba813eb447b823c91f,Doting-Fiction,,,,0x48671d8cc40f1d1cd9fe227f6d7858cb25a4e0fc73b55b70d5616a91505e6ac6,2025-09-03 14:05:02+00:00


In [70]:
# Compute per-wallet UP positions at a cutoff timestamp
from typing import Optional
import pandas as pd, math, datetime

UP_LABELS = {"Yes", "YES", "Up", "UP"}
DOWN_LABELS = {"No", "NO", "Down", "DOWN"}

if 'dt' not in df.columns:
    raise ValueError("DataFrame df must contain 'dt' datetime column; re-run load cell.")

required_cols = {'proxyWallet', 'side', 'size', 'price', 'name'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required trade columns: {missing}")

if 'outcome' not in df.columns and 'outcomeIndex' not in df.columns:
    raise ValueError("Need either 'outcome' or 'outcomeIndex' column to classify trades.")

work = df.copy()

def classify_outcome(row):
    if 'outcome' in row and isinstance(row['outcome'], str):
        o = row['outcome']
        if o in UP_LABELS:
            return 'UP'
        if o in DOWN_LABELS:
            return 'DOWN'

work['__class'] = work.apply(classify_outcome, axis=1)
unknown_ct = (work['__class'] == 'UNKNOWN').sum()
if unknown_ct:
    print(f"Warning: {unknown_ct} trades could not be classified (tagged UNKNOWN). They are ignored.")
work = work[work['__class'] != 'UNKNOWN']

def compute_deltas(row):
    side = row['side']
    size = float(row['size'])
    price = float(row['price'])
    cls = row['__class']
    if cls == 'UP':
        if side == 'BUY':
            return size, -price * size
        else:
            return -size, +price * size
    else:  # DOWN
        price_up = 1.0 - price
        if side == 'BUY':
            return -size, +price_up * size
        else:
            return +size, -price_up * size

work[['delta_shares_up','cash_flow']] = work.apply(lambda r: pd.Series(compute_deltas(r)), axis=1)

agg = work.groupby('proxyWallet', as_index=False).agg(
    name=('name','first'),
    shares_up=('delta_shares_up','sum'),
    cash_flow=('cash_flow','sum'),
    trade_count=('delta_shares_up','count')
)

def avg_entry(row):
    if row['shares_up'] > 0:
        return (-row['cash_flow']) / row['shares_up'] if row['shares_up'] != 0 else math.nan
    return math.nan

agg['avg_entry_price_est'] = agg.apply(avg_entry, axis=1)
agg.sort_values('shares_up', ascending=False, inplace=True)
agg.reset_index(drop=True, inplace=True)

agg.head(25)

,proxyWallet,name,shares_up,cash_flow,trade_count,avg_entry_price_est
0,0x80f8bb68361f2a2ff2709e59a544a98ab2946713,SergioVenezia,3.831413e+06,-1.270255e+06,489,0.331537
1,0x0bddc0e587d6c6353fd2f9ef630bc216ed0fc8ad,hellova,4.022017e+05,-1.365920e+05,490,0.339611
2,0x48354af7e126e205ba9d2bb74b7f10d552e38f59,RickBlaine,1.423333e+05,-5.111298e+04,490,0.359108
3,0x5df52b96990dc5079f30870ef32358f13afcc44c,CajetanCryptoTrading-Hour1,1.026900e+05,-7.716420e+04,4401,0.751429
4,0x9b5b249c70f434f88fdc4cdc9a3be036b04d11f3,,9.475312e+04,-1.091005e+05,4448,1.151418
5,0x5339c9da744ff14e978081f99c2309c4d5529da3,JohnReacher,5.375537e+04,-2.740096e+04,490,0.509734
6,0x104a733d68fa06e916daa5ba7ab9f464ace2d487,1749177280,4.890000e+04,-2.239620e+04,489,0.458000
7,0x092cb1e22e35ddabc8fa321e33bc0c4c151097de,yasnovidka,4.154500e+04,-1.445442e+04,979,0.347922
8,0x51196e2b4b60547cc6e8bd1b6527c60f2bb7e038,xmsx,3.905324e+04,-1.402222e+04,490,0.359054
9,0x3a972c07d05c81e799053d27bb7ef05e6eda7115,0x3A972C07d05C81E799053D27bb7eF05e6EdA7115-1756218819276,3.667500e+04,-2.720473e+04,489,0.741779


In [ ]:
# start = datetime.datetime(2025, 9, 3, 14, 20, 0, tzinfo=datetime.timezone.utc)
# end = datetime.datetime(2025, 9, 3, 14, 40, 0, tzinfo=datetime.timezone.utc)

# df = df[(df['dt'] >= start) & (df['dt'] <= end)]

In [90]:
# Plot btcPrice from estimator JSONL vs Polymarket UP price + model estimate (cropped to estimator time range)
import json, pathlib, pandas as pd, plotly.graph_objects as go, datetime, re
from typing import List, Dict, Any

# Ensure trades df exists
if 'df' not in globals():
    raise ValueError("Trades DataFrame `df` not found. Run the load trades cell first.")

plot_df = df.copy()
for col in ['price','size','dt']:
    if col not in plot_df.columns:
        raise ValueError(f'Missing column {col} in trades DataFrame.')

# Classify outcomes if needed
if '__class' not in plot_df.columns:
    UP_LABELS = {"Yes", "YES", "Up", "UP"}
    DOWN_LABELS = {"No", "NO", "Down", "DOWN"}
    def _cls(row):
        if 'outcome' in row and isinstance(row['outcome'], str):
            if row['outcome'] in UP_LABELS: return 'UP'
            if row['outcome'] in DOWN_LABELS: return 'DOWN'
        if 'outcomeIndex' in row:
            try:
                return 'UP' if int(row['outcomeIndex']) == 0 else 'DOWN'
            except Exception:
                return 'UNKNOWN'
        return 'UNKNOWN'
    plot_df['__class'] = plot_df.apply(_cls, axis=1)

plot_df = plot_df[plot_df['__class'] != 'UNKNOWN'].copy()
plot_df['up_price'] = plot_df.apply(lambda r: float(r['price']) if r['__class'] == 'UP' else 1.0 - float(r['price']), axis=1)
plot_df['size'] = plot_df['size'].astype(float)

# --- Load estimator file (provided path) ---
estimator_dir = pathlib.Path('/Users/vobornij/projects/polymarket/')
base_id = None
if 'market' in globals():
    base_id = re.sub(r'^0x','', str(market))

est_df = pd.DataFrame()
rows = []
with (estimator_dir / estimator_path).open() as f:
    for line in f:
        line=line.strip()
        if not line: continue
        try:
            rows.append(json.loads(line))
        except Exception:
            pass
if rows:
    est_df = pd.DataFrame(rows)
    # Timestamp
    tcol = 'timestamp' if 'timestamp' in est_df.columns else None
    if tcol is not None:
        est_df['dt'] = pd.to_datetime(est_df[tcol], utc=True, errors='coerce')
    else:
        for alt in ['time','ts','datetime','dt']:
            if alt in est_df.columns:
                est_df['dt'] = pd.to_datetime(est_df[alt], utc=True, errors='coerce')
                break
    est_df = est_df.dropna(subset=['dt'])
    # Numeric conversion for new format fields
    numeric_cols = ['btcPrice','target','estimate','seasonalSigma','sigmaEwmShrunkNormalized','sigmaEwmShrunkNormalizedFactor','backbone']
    for c in numeric_cols:
        if c in est_df.columns:
            est_df[c] = pd.to_numeric(est_df[c], errors='coerce')
    keep_cols = [c for c in ['dt','btcPrice','estimate','target','seasonalSigma','sigmaEwmShrunkNormalized','sigmaEwmShrunkNormalizedFactor','backbone'] if c in est_df.columns]
    est_df = est_df[keep_cols].sort_values('dt')

# Crop trades to estimator time range if estimator data present
if not est_df.empty:
    est_start, est_end = est_df['dt'].min(), est_df['dt'].max()
    plot_df = plot_df[(plot_df['dt'] >= est_start) & (plot_df['dt'] <= est_end)].copy()

# Recompute per-second weighted average after potential cropping
if not plot_df.empty:
    plot_df['dt_sec'] = plot_df['dt'].dt.floor('s')
    agg_up = (plot_df.groupby('dt_sec')
              .apply(lambda g: (g['up_price'] * g['size']).sum() / g['size'].sum(), include_groups=False)
              .rename('up_wap')
              .to_frame()
              .reset_index())
else:
    agg_up = pd.DataFrame(columns=['dt_sec','up_wap'])

In [91]:
est_df.iloc[0]

dt                                2025-09-03 14:00:00+00:00
btcPrice                                          111475.07
estimate                                           0.499478
target                                            111475.07
seasonalSigma                                      0.000624
sigmaEwmShrunkNormalized                           1.000768
sigmaEwmShrunkNormalizedFactor                      0.00073
backbone                                                0.0
Name: 0, dtype: object

In [92]:
# Updated plot for new estimator schema (no scalingFactor) + added volatility traces if present
# Primary y-axis now centered around target so target line aligns with 0.5 on secondary probability axis.
# Restored rich hover functionality with unified crosshair and per-trace custom hover templates.
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

if 'est_df' not in globals() or est_df.empty:
    raise ValueError('est_df empty - run estimator load cell first.')

fig = make_subplots(specs=[[{"secondary_y": True}]])

hover_time_fmt = '%Y-%m-%d %H:%M:%S'

# BTC price
fig.add_scatter(
    x=est_df['dt'], y=est_df['btcPrice'], name='BTC Price', secondary_y=False,
    line=dict(color='gray'),
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>BTC: %{y:.2f}<extra>BTC Price</extra>'
)

# Target horizontal (record for centering)
targ = est_df['target'].dropna().iloc[0]
fig.add_scatter(
    x=est_df['dt'], y=[targ]*len(est_df), name='Target', secondary_y=False,
    line=dict(color='black', dash='dash'),
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Target: '+ '{:.2f}'.format(targ) +'<extra>Target</extra>'
)

# Probabilities: aggregated WAP & raw trades
fig.add_scatter(
    x=agg_up['dt_sec'], y=agg_up['up_wap'], name='UP WAP', mode='lines', line=dict(color='royalblue'), secondary_y=True,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>UP WAP: %{y:.3f}<extra>UP WAP</extra>'
)

fig.add_scatter(
    x=plot_df['dt'], y=plot_df['up_price'], name='UP Trades', mode='markers',
    marker=dict(size=4, color='royalblue', opacity=0.3), secondary_y=True,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>UP Trade: %{y:.3f}<br>Size: %{customdata[0]:.2f}<extra>Trade</extra>',
    customdata=np.stack([plot_df['size'].values], axis=-1)
)

# Model estimate probability
fig.add_scatter(
    x=est_df['dt'], y=est_df['estimate'], name='Model Estimate', line=dict(color='orange'), secondary_y=True,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Estimate: %{y:.3f}<extra>Model</extra>'
)

# Vol / variance style fields on primary axis
bb_sigma = np.sqrt(est_df['backbone'].clip(lower=0))
fig.add_scatter(
    x=est_df['dt'], y=bb_sigma, name='Backbone σ', line=dict(color='purple', dash='dot'), secondary_y=False,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Backbone σ: %{y:.4f}<extra>Backbone σ</extra>'
)

fig.add_scatter(
    x=est_df['dt'], y=est_df['seasonalSigma'], name='Seasonal σ', line=dict(color='green', dash='dot'), secondary_y=False,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>Seasonal σ: %{y:.4f}<extra>Seasonal σ</extra>'
)

fig.add_scatter(
    x=est_df['dt'], y=est_df['sigmaEwmShrunkNormalized'], name='σ EWM Shrunk Norm', line=dict(color='teal', dash='dashdot'), secondary_y=False,
    hovertemplate='Time: %{x|'+hover_time_fmt+'}<br>σ EWM Shrunk Norm: %{y:.4f}<extra>σ EWM Shrunk Norm</extra>'
)

# Axis styling
fig.update_yaxes(title_text='BTC Price / σ', secondary_y=False)
fig.update_yaxes(title_text='Probability', range=[0,1], secondary_y=True)

# Center primary y-axis around target so its midpoint aligns with probability 0.5.
if targ is not None and 'btcPrice' in est_df.columns:
    price_series = est_df['btcPrice']
    max_dev = (price_series - targ).abs().max()
    if max_dev == 0:
        max_dev = max(abs(targ)*0.01, 1)
    pad = max_dev * 0.05
    fig.update_yaxes(range=[targ - max_dev - pad, targ + max_dev + pad], secondary_y=False)
else:
    if 'btcPrice' in est_df.columns:
        pmin, pmax = est_df['btcPrice'].min(), est_df['btcPrice'].max()
        pad = (pmax - pmin) * 0.05 if pmax > pmin else (pmax * 0.05 if pmax else 1)
        fig.update_yaxes(range=[pmin - pad, pmax + pad], secondary_y=False)

fig.update_layout(height=650, title='Trade vs Estimator', hovermode='x unified')
fig.show(renderer='browser')

In [93]:
trades2 = df.copy()[['dt', 'side', 'outcome', 'name', 'price', 'size', 'proxyWallet']]

In [94]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

UP_LABELS = {"Yes", "YES", "Up", "UP"}

# up_price: normalize to UP outcome probability
trades2['up_price'] = trades2.apply(
    lambda r: float(r['price']) if r.get('outcome') in UP_LABELS else 1.0 - float(r['price']),
    axis=1
)

# up_side: perspective of the UP outcome (BUY means increasing UP exposure)
def _up_side(r):
    side = r.get('side')
    if r.get('outcome') in UP_LABELS:
        return side  # already oriented to UP
    # Outcome is DOWN: a BUY on DOWN reduces UP exposure => treat as SELL (and vice‑versa)
    if side == 'BUY':
        return 'SELL'
    if side == 'SELL':
        return 'BUY'
    return side

trades2['up_side'] = trades2.apply(_up_side, axis=1)

trades2[['dt', 'up_side', 'up_price', 'size', 'name', 'proxyWallet', 'side', 'outcome']].tail(10)

,dt,up_side,up_price,size,name,proxyWallet,side,outcome
377,2025-09-03 14:39:50+00:00,BUY,0.94,100.000000,0xf247584e41117bbBe4Cc06E4d2C95741792a5216-1742469835200,0xf247584e41117bbbe4cc06e4d2c95741792a5216,SELL,Down
372,2025-09-03 14:39:52+00:00,SELL,0.91,5.330000,zp312,0x754995ab57b498ae81f0a606b07ad281c26e0e7e,SELL,Up
371,2025-09-03 14:39:52+00:00,BUY,0.95,625.150000,hembag,0x9155e8cf81a3fb557639d23d43f1528675bcfcad,BUY,Up
370,2025-09-03 14:39:54+00:00,BUY,0.95,5.000000,costunan,0x4ef0194e8cfd5617972665826f402836ac5f15a0,BUY,Up
369,2025-09-03 14:39:54+00:00,BUY,0.95,5.000000,costunan,0x4ef0194e8cfd5617972665826f402836ac5f15a0,BUY,Up
368,2025-09-03 14:39:54+00:00,BUY,0.95,5.000000,costunan,0x4ef0194e8cfd5617972665826f402836ac5f15a0,BUY,Up
367,2025-09-03 14:39:54+00:00,BUY,0.95,163.000000,costunan,0x4ef0194e8cfd5617972665826f402836ac5f15a0,BUY,Up
366,2025-09-03 14:39:58+00:00,SELL,0.93,26.040000,Auron1,0x31518ca69ba78e6e00cf5b539c8713e8753174e7,SELL,Up
365,2025-09-03 14:39:58+00:00,BUY,0.96,298.604165,FENDI969,0x18fab8902f6a9920d253062daa2d82e870fdbd9d,BUY,Up
364,2025-09-03 14:40:00+00:00,BUY,0.95,76.400000,hembag,0x9155e8cf81a3fb557639d23d43f1528675bcfcad,BUY,Up
